**做多交易策略：**

1. 寻找正在创下20天高点的货币对。
2. 观察该货币对当天或次日反转，创下两天低点。
3. 如果该货币对在创下两天低点后的三天内突破20天高点，则买入。
4. 初始止损设置在两天低点下方的几个点。
5. 当价格上涨到与你的风险相同的点数时，平掉一半仓位，并将剩余仓位的止损移动到盈亏平衡点。
6. 对剩余仓位设置移动止损（追踪止损）。

In [18]:
import pandas as pd
from backend.data_center.kline_data.kline_data_collector import *
tv = KlineDataCollector()
file_abspath = tv.get_abspath(symbol='BTC', interval=Interval.in_daily)
df = pd.read_csv(f"{file_abspath}")
df

,datetime,symbol,open,high,low,close,volume
0,2017-08-17 08:00:00,Binance:BTCUSDT,4261.48,4485.39,4200.74,4285.08,795.150377
1,2017-08-18 08:00:00,Binance:BTCUSDT,4285.08,4371.52,3938.77,4108.37,1199.888264
2,2017-08-19 08:00:00,Binance:BTCUSDT,4108.37,4184.69,3850.00,4139.98,381.309763
3,2017-08-20 08:00:00,Binance:BTCUSDT,4120.98,4211.08,4032.62,4086.29,467.083022
4,2017-08-21 08:00:00,Binance:BTCUSDT,4069.13,4119.62,3911.79,4016.00,691.743060
...,...,...,...,...,...,...,...
2604,2024-10-03 08:00:00,Binance:BTCUSDT,60649.27,61477.19,59828.11,60752.71,26224.819320
2605,2024-10-04 08:00:00,Binance:BTCUSDT,60752.72,62484.85,60459.90,62086.00,21299.146910
2606,2024-10-05 08:00:00,Binance:BTCUSDT,62086.00,62370.56,61689.26,62058.00,7848.591930
2607,2024-10-06 08:00:00,Binance:BTCUSDT,62058.01,62975.00,61798.97,62819.91,8961.253090


In [39]:
# df['is_20d_high'] = df['high'] == df['high'].expanding(min_periods=3).max()

df['is_20d_high'] = (df['high'] == df['high'].rolling(window=20, min_periods=1).max())
df['is_2d_low'] = (df['low'] == df['low'].rolling(window=3, min_periods=1).min())
df[df['is_20d_high']==True]

,datetime,symbol,open,high,low,close,volume,is_20d_high,20d_high,is_2d_low
0,2017-08-17 08:00:00,Binance:BTCUSDT,4261.48,4485.39,4200.74,4285.08,795.150377,True,4485.39,True
12,2017-08-29 08:00:00,Binance:BTCUSDT,4353.65,4625.85,4313.55,4587.48,603.545028,True,4625.85,False
13,2017-08-30 08:00:00,Binance:BTCUSDT,4564.52,4647.51,4416.01,4555.14,808.468771,True,4647.51,False
14,2017-08-31 08:00:00,Binance:BTCUSDT,4555.14,4745.42,4555.14,4724.89,556.956802,True,4745.42,False
15,2017-09-01 08:00:00,Binance:BTCUSDT,4689.89,4885.55,4654.88,4834.91,560.666366,True,4885.55,False
...,...,...,...,...,...,...,...,...,...,...
2591,2024-09-20 08:00:00,Binance:BTCUSDT,62948.00,64133.32,62350.00,63201.05,25466.377940,True,64133.32,False
2594,2024-09-23 08:00:00,Binance:BTCUSDT,63578.76,64745.88,62538.75,63339.99,24078.052870,True,64745.88,False
2596,2024-09-25 08:00:00,Binance:BTCUSDT,64262.70,64817.99,62947.08,63152.01,17813.111680,True,64817.99,False
2597,2024-09-26 08:00:00,Binance:BTCUSDT,63152.01,65839.00,62670.00,65173.99,28373.305930,True,65839.00,True


In [43]:
# df['20d_high'] = df['high'].rolling(window=20, min_periods=1).max()
df['have_20day_breakout'] = (
    df['is_20d_high'].rolling(window=5).max().shift(1) == True
) & (
    df['is_2d_low'].rolling(window=2).max().shift(1) == True
) & (
    df['is_20d_high']==True
)
df[df['have_20day_breakout']]
# 2024-03-12
# 2024-09-27

,datetime,symbol,open,high,low,close,volume,is_20d_high,20d_high,is_2d_low,have_20day_breakout
13,2017-08-30 08:00:00,Binance:BTCUSDT,4564.52,4647.51,4416.01,4555.14,808.468771,True,4647.51,False,True
64,2017-10-20 08:00:00,Binance:BTCUSDT,5683.31,6110.00,5600.00,6010.01,1972.977220,True,6110.00,False,True
74,2017-10-30 08:00:00,Binance:BTCUSDT,6133.01,6248.68,6030.00,6120.50,1473.687043,True,6248.68,False,True
83,2017-11-08 08:00:00,Binance:BTCUSDT,7070.00,7770.02,6651.00,7303.00,2822.298802,True,7770.02,True,True
100,2017-11-25 08:00:00,Binance:BTCUSDT,8138.99,8734.78,8090.00,8700.01,4292.623682,True,8734.78,False,True
101,2017-11-26 08:00:00,Binance:BTCUSDT,8700.04,9350.00,8604.72,9128.02,4147.380237,True,9350.00,False,True
116,2017-12-11 08:00:00,Binance:BTCUSDT,14901.08,17470.00,14901.08,16560.02,11822.314528,True,17470.00,False,True
120,2017-12-15 08:00:00,Binance:BTCUSDT,16339.99,17991.00,16300.06,17516.81,9181.486777,True,17991.00,False,True
340,2018-07-23 08:00:00,Binance:BTCUSDT,7394.78,7833.00,7375.00,7721.01,41646.346671,True,7833.00,False,True
380,2018-09-01 08:00:00,Binance:BTCUSDT,7011.21,7275.00,7008.74,7200.01,40323.414093,True,7275.00,False,True
